In [1]:
import pandas as pd 
import numpy as np 
import torchvision

In [2]:
from torch.utils.data import DataLoader, Dataset
class NestedImageFolder(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_to_idx = {}

        # Walk through the root directory and get all images
        for idx, class_name in enumerate(os.listdir(root_dir)):
            class_dir = os.path.join(root_dir, class_name)
            if os.path.isdir(class_dir):
                self.class_to_idx[class_name] = idx
                for subdir, _, files in os.walk(class_dir):
                    for file in files:
                        if file.endswith(('jpg', 'jpeg', 'png')):  # add other extensions if needed
                            self.image_paths.append(os.path.join(subdir, file))
                            self.labels.append(idx)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
from PIL import Image

base_dir = os.getcwd()

# Construct the training and validation directory paths
train_dir = os.path.join(base_dir, 'Dataset', 'training')
valid_dir = os.path.join(base_dir, 'Dataset', 'validation')

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a common size
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the images
])

# Load the training and validation datasets
train_dataset = NestedImageFolder(root_dir=train_dir, transform=transform)
valid_dataset = NestedImageFolder(root_dir=valid_dir, transform=transform)

# Create DataLoaders to handle batching and shuffling
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

# Get the class names (labels) from the dataset
class_names = train_dataset.class_to_idx.keys()
print(f"Classes: {class_names}")

# Example: Accessing the dataset
train_iter = iter(train_loader)
images, labels = next(train_iter)
print(f"Batch size: {images.size(0)}")
print(f"Image dimensions: {images.size()[1:]}")  # [channels, height, width]
print(f"Labels: {labels}")


In [ ]:
from collections import Counter

class_counts = Counter(train_dataset.labels)  
print(f"Training set class distribution: {class_counts}")

valid_class_counts = Counter(valid_dataset.labels)
print(f"Validation set class distribution: {valid_class_counts}")


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# Get some random training images
dataiter = iter(train_loader)
images, labels = next(dataiter)

# Show images
imshow(torchvision.utils.make_grid(images))


In [ ]:
import torch
from torchvision import models
from torchvision.models import ResNet18_Weights

import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = models.resnet18(weights='DEFAULT')
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

print("Training complete")